In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import random
import time
import warnings

# 忽略 FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning)



# Function to keep search agent positions within bounds
def boundary(position, lb, ub):
    return np.clip(position, lb, ub)

# Define the cache outside the function
cache = {}

# Fitness function for XGBoost
def fitness_xgboost(solution, X1, y1, Xt, yt, w1=1, w2=1):
    key = tuple(solution)  # Cache key
    if key in cache:
        return cache[key]

    n_estimators = int(solution[0])
    max_depth = int(solution[1])
    learning_rate = solution[2]
    model = XGBRegressor(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, 
                         objective='reg:squarederror', random_state=42, n_jobs=-1)
    model.fit(X1, y1)
    predictions = model.predict(Xt)
    mse = mean_squared_error(yt, predictions)
    r2 = r2_score(yt, predictions)

    fitness_value = w1 * mse - w2 * r2  # Weighted MSE and R2 for optimization
    cache[key] = fitness_value  # Store result in cache
    return fitness_value

# Fitness function for RandomForest
def fitness_random_forest(solution, X1, y1, Xt, yt):
    n_estimators = int(solution[0])
    max_depth = int(solution[1])
    min_samples_split = int(solution[2])
    model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split, random_state=42)
    model.fit(X1, y1)
    predictions = model.predict(Xt)
    mse = mean_squared_error(yt, predictions)
    return mse  # Optimize MSE

# Harris Hawks Optimization Algorithm (HHO) for XGBoost
def hho_for_xgboost(SearchAgents_no, Max_iter, X1, y1, Xt, yt, w1=1, w2=1):
    dim = 3  # Dimension of the optimization problem (n_estimators, max_depth, learning_rate)
    Leader_pos = np.zeros(dim)  # Leader position (best solution)
    Leader_score = float('inf')  # Leader's fitness score

    # Boundaries of the search space for XGBoost
    lb = np.array([50, 1, 0.01])  # Lower bounds
    ub = np.array([500, 50, 0.5])  # Upper bounds

    # Initialize the location of the search agents
    Positions = np.zeros((SearchAgents_no, dim))
    for i in range(SearchAgents_no):
        Positions[i, :] = np.array([ 
            random.randint(int(lb[0]), int(ub[0])), 
            random.randint(int(lb[1]), int(ub[1])), 
            random.uniform(lb[2], ub[2]) 
        ])

    Convergence_curve = np.zeros(Max_iter)

    for t in range(Max_iter):
        E1 = 2 * (1 - t / Max_iter)  # Energy parameter decreases linearly

        for i in range(SearchAgents_no):
            # Calculate fitness of the current agent
            fit = fitness_xgboost(Positions[i, :], X1, y1, Xt, yt, w1, w2)

            # Update the leader if the current agent is better
            if fit < Leader_score:
                Leader_score = fit
                Leader_pos = Positions[i, :].copy()

        for i in range(SearchAgents_no):
            E0 = 2 * random.random() - 1  # Random energy
            E = E1 * E0  # Current energy of the prey
            Q = random.random()  # Jumping strength
            J = 2 * (1 - random.random())  # Randomization parameter

            if abs(E) >= 1:  # Exploration phase
                rand_idx = random.randint(0, SearchAgents_no - 1)
                X_rand = Positions[rand_idx, :]
                Positions[i, :] = X_rand - random.random() * abs(X_rand - 2 * random.random() * Positions[i, :])
            else:  # Exploitation phase
                if Q < 0.5:  # Soft besiege
                    Positions[i, :] = Leader_pos - E * abs(J * Leader_pos - Positions[i, :])
                else:  # Hard besiege
                    Positions[i, :] = (Leader_pos - Positions[i, :]) - E * abs(J * Leader_pos - Positions[i, :])

            # Ensure agents stay within the search space
            Positions[i, :] = boundary(Positions[i, :], lb, ub)

            # Calculate fitness of the updated agent
            fit = fitness_xgboost(Positions[i, :], X1, y1, Xt, yt, w1, w2)

            # Update the leader if the updated agent is better
            if fit < Leader_score:
                Leader_score = fit
                Leader_pos = Positions[i, :].copy()

        Convergence_curve[t] = Leader_score  # Record the best fitness

    return Leader_pos, Convergence_curve

# Harris Hawks Optimization Algorithm (HHO) for Random Forest
def hho_for_random_forest(SearchAgents_no, Max_iter, X1, y1, Xt, yt):
    dim = 3  # Dimension of the optimization problem (n_estimators, max_depth, min_samples_split)
    Leader_pos = np.zeros(dim)  # Leader position (best solution)
    Leader_score = float('inf')  # Leader's fitness score

    # Boundaries of the search space for RandomForest
    lb = np.array([50, 1, 1])  # Lower bounds for n_estimators, max_depth, min_samples_split
    ub = np.array([500, 50, 50])  # Upper bounds

    # Initialize the location of the search agents
    Positions = np.zeros((SearchAgents_no, dim))
    for i in range(SearchAgents_no):
        Positions[i, :] = np.array([ 
            random.randint(lb[0], ub[0]), 
            random.randint(lb[1], ub[1]), 
            random.randint(lb[2], ub[2]) 
        ])

    Convergence_curve = np.zeros(Max_iter)

    for t in range(Max_iter):
        E1 = 2 * (1 - t / Max_iter)  # Energy parameter decreases linearly

        for i in range(SearchAgents_no):
            # Calculate fitness of the current agent
            fit = fitness_random_forest(Positions[i, :], X1, y1, Xt, yt)

            # Update the leader if the current agent is better
            if fit < Leader_score:
                Leader_score = fit
                Leader_pos = Positions[i, :].copy()

        for i in range(SearchAgents_no):
            E0 = 2 * random.random() - 1  # Random energy
            E = E1 * E0  # Current energy of the prey
            Q = random.random()  # Jumping strength
            J = 2 * (1 - random.random())  # Randomization parameter

            if abs(E) >= 1:  # Exploration phase
                rand_idx = random.randint(0, SearchAgents_no - 1)
                X_rand = Positions[rand_idx, :]
                Positions[i, :] = X_rand - random.random() * abs(X_rand - 2 * random.random() * Positions[i, :])
            else:  # Exploitation phase
                if Q < 0.5:  # Soft besiege
                    Positions[i, :] = Leader_pos - E * abs(J * Leader_pos - Positions[i, :])
                else:  # Hard besiege
                    Positions[i, :] = (Leader_pos - Positions[i, :]) - E * abs(J * Leader_pos - Positions[i, :])

            # Ensure agents stay within the search space
            Positions[i, :] = boundary(Positions[i, :], lb, ub)

            # Calculate fitness of the updated agent
            fit = fitness_random_forest(Positions[i, :], X1, y1, Xt, yt)

            # Update the leader if the updated agent is better
            if fit < Leader_score:
                Leader_score = fit
                Leader_pos = Positions[i, :].copy()

        Convergence_curve[t] = Leader_score  # Record the best fitness

    return Leader_pos, Convergence_curve


def get_best_params_from_leader(Leader_pos, model_type):
    if model_type == 'xgboost':
        return {
            'n_estimators': int(Leader_pos[0]),
            'max_depth': int(Leader_pos[1]),
            'learning_rate': Leader_pos[2]
        }
    elif model_type == 'random_forest':
        return {
            'n_estimators': int(Leader_pos[0]),
            'max_depth': int(Leader_pos[1]),
            'min_samples_split': int(Leader_pos[2])
        }

# 固定随机种子，保证每次运行的结果一致
np.random.seed(42)
# 加载数据
df = pd.read_excel('soil thickness 环境因子.xlsx')

# 分离输入和输出
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# 定义两组特征列
factors_group_1 = ['NDVI', 'TWI', 'ASP', 'LU', 'P', 'SI', 'C']   # 第一组特征
factors_group_2 = ['NDVI', 'TWI', 'ASP', 'LU', 'SSP', 'SSSI', 'C'] # 第二组特征

# 初始化XGBoost和RandomForest的结果列表
xgb_results_group_1 = []
xgb_results_group_2 = []
rf_results_group_1 = []
rf_results_group_2 = []


# 设置迭代次数和群体大小
SearchAgents_no = 50
Max_iter = 20
# Perform 100 training and testing of HHO-RF models

for iteration in range(100):
    # 每次迭代都重新划分数据集，确保两个模型使用相同的训练集和测试集
    train_data, test_data, train_target, test_target = train_test_split(X, y, train_size=0.7, random_state=iteration)
    # Perform HHO optimization
    


    # 提取不同特征组的数据
    train_data_group_1 = train_data[factors_group_1]
    test_data_group_1 = test_data[factors_group_1]
    
    train_data_group_2 = train_data[factors_group_2]
    test_data_group_2 = test_data[factors_group_2]

   
    # 运行HHO优化算法并获取最佳参数
    # 对第一组特征进行XGBoost优化
    xgb_params_group_1_optimized, _ = hho_for_xgboost(SearchAgents_no, Max_iter, train_data_group_1, train_target, test_data_group_1, test_target)
    xgb_params_group_1_optimized = get_best_params_from_leader(xgb_params_group_1_optimized, 'xgboost')
    
    # 对第二组特征进行XGBoost优化
    xgb_params_group_2_optimized, _ = hho_for_xgboost(SearchAgents_no, Max_iter, train_data_group_2, train_target, test_data_group_2, test_target)
    xgb_params_group_2_optimized = get_best_params_from_leader(xgb_params_group_2_optimized, 'xgboost')
    
    # 对第一组特征进行RandomForest优化
    rf_params_group_1_optimized, _ = hho_for_random_forest(SearchAgents_no, Max_iter, train_data_group_1, train_target, test_data_group_1, test_target)
    rf_params_group_1_optimized = get_best_params_from_leader(rf_params_group_1_optimized, 'random_forest')
    
    # 对第二组特征进行RandomForest优化
    rf_params_group_2_optimized, _ = hho_for_random_forest(SearchAgents_no, Max_iter, train_data_group_2, train_target, test_data_group_2, test_target)
    rf_params_group_2_optimized = get_best_params_from_leader(rf_params_group_2_optimized, 'random_forest')

    
    # 训练 XGBoost（第一组特征）
    model_xgb_group_1 = XGBRegressor(**xgb_params_group_1_optimized)
    model_xgb_group_1.fit(train_data_group_1, train_target)
    
    # 训练 XGBoost（第二组特征）
    model_xgb_group_2 = XGBRegressor(**xgb_params_group_2_optimized)
    model_xgb_group_2.fit(train_data_group_2, train_target)
    
    # 训练 RandomForest（第一组特征）
    model_rf_group_1 = RandomForestRegressor(**rf_params_group_1_optimized)
    model_rf_group_1.fit(train_data_group_1, train_target)

    # 训练 RandomForest（第二组特征）
    model_rf_group_2 = RandomForestRegressor(**rf_params_group_2_optimized)
    model_rf_group_2.fit(train_data_group_2, train_target)

    # 预测和性能评估（RandomForest）
    train_pred_rf_group_1 = model_rf_group_1.predict(train_data_group_1)
    test_pred_rf_group_1 = model_rf_group_1.predict(test_data_group_1)

    train_pred_rf_group_2 = model_rf_group_2.predict(train_data_group_2)
    test_pred_rf_group_2 = model_rf_group_2.predict(test_data_group_2)

    # 预测和性能评估（XGBoost）
    train_pred_xgb_group_1 = model_xgb_group_1.predict(train_data_group_1)
    test_pred_xgb_group_1 = model_xgb_group_1.predict(test_data_group_1)

    train_pred_xgb_group_2 = model_xgb_group_2.predict(train_data_group_2)
    test_pred_xgb_group_2 = model_xgb_group_2.predict(test_data_group_2)

    # 计算MSE和RMSE
    train_mse_rf_group_1 = mean_squared_error(train_target, train_pred_rf_group_1)
    test_mse_rf_group_1 = mean_squared_error(test_target, test_pred_rf_group_1)

    train_mse_rf_group_2 = mean_squared_error(train_target, train_pred_rf_group_2)
    test_mse_rf_group_2 = mean_squared_error(test_target, test_pred_rf_group_2)

    train_mse_xgb_group_1 = mean_squared_error(train_target, train_pred_xgb_group_1)
    test_mse_xgb_group_1 = mean_squared_error(test_target, test_pred_xgb_group_1)

    train_mse_xgb_group_2 = mean_squared_error(train_target, train_pred_xgb_group_2)
    test_mse_xgb_group_2 = mean_squared_error(test_target, test_pred_xgb_group_2)

    # 计算RMSE
    train_rmse_rf_group_1 = np.sqrt(train_mse_rf_group_1)
    test_rmse_rf_group_1 = np.sqrt(test_mse_rf_group_1)

    train_rmse_rf_group_2 = np.sqrt(train_mse_rf_group_2)
    test_rmse_rf_group_2 = np.sqrt(test_mse_rf_group_2)

    train_rmse_xgb_group_1 = np.sqrt(train_mse_xgb_group_1)
    test_rmse_xgb_group_1 = np.sqrt(test_mse_xgb_group_1)

    train_rmse_xgb_group_2 = np.sqrt(train_mse_xgb_group_2)
    test_rmse_xgb_group_2 = np.sqrt(test_mse_xgb_group_2)

    # 计算R2
    train_r2_rf_group_1 = r2_score(train_target, train_pred_rf_group_1)
    test_r2_rf_group_1 = r2_score(test_target, test_pred_rf_group_1)

    train_r2_rf_group_2 = r2_score(train_target, train_pred_rf_group_2)
    test_r2_rf_group_2 = r2_score(test_target, test_pred_rf_group_2)

    train_r2_xgb_group_1 = r2_score(train_target, train_pred_xgb_group_1)
    test_r2_xgb_group_1 = r2_score(test_target, test_pred_xgb_group_1)

    train_r2_xgb_group_2 = r2_score(train_target, train_pred_xgb_group_2)
    test_r2_xgb_group_2 = r2_score(test_target, test_pred_xgb_group_2)
    # 输出每次迭代的结果


    # 将每次迭代的结果保存到列表
    rf_results_group_1.append({
        'iteration': iteration + 1,
        'train_mse': train_mse_rf_group_1, 'test_mse': test_mse_rf_group_1,
        'train_rmse': train_rmse_rf_group_1, 'test_rmse': test_rmse_rf_group_1,
        'train_r2': train_r2_rf_group_1, 'test_r2': test_r2_rf_group_1,
        'n_estimators': rf_params_group_1_optimized['n_estimators'],
        'max_depth': rf_params_group_1_optimized['max_depth'],
        'min_samples_split': rf_params_group_1_optimized['min_samples_split']
    })
    rf_results_group_2.append({
        'iteration': iteration + 1,
        'train_mse': train_mse_rf_group_2, 'test_mse': test_mse_rf_group_2,
        'train_rmse': train_rmse_rf_group_2, 'test_rmse': test_rmse_rf_group_2,
        'train_r2': train_r2_rf_group_2, 'test_r2': test_r2_rf_group_2,
        'n_estimators': rf_params_group_2_optimized['n_estimators'],
        'max_depth': rf_params_group_2_optimized['max_depth'],
        'min_samples_split': rf_params_group_2_optimized['min_samples_split']
    })

    xgb_results_group_1.append({
        'iteration': iteration + 1,
        'train_mse': train_mse_xgb_group_1, 'test_mse': test_mse_xgb_group_1,
        'train_rmse': train_rmse_xgb_group_1, 'test_rmse': test_rmse_xgb_group_1,
        'train_r2': train_r2_xgb_group_1, 'test_r2': test_r2_xgb_group_1,
        'n_estimators': xgb_params_group_1_optimized['n_estimators'],
        'max_depth': xgb_params_group_1_optimized['max_depth'],
        'learning_rate': xgb_params_group_1_optimized['learning_rate']
    })
    xgb_results_group_2.append({
        'iteration': iteration + 1,
        'train_mse': train_mse_xgb_group_2, 'test_mse': test_mse_xgb_group_2,
        'train_rmse': train_rmse_xgb_group_2, 'test_rmse': test_rmse_xgb_group_2,
        'train_r2': train_r2_xgb_group_2, 'test_r2': test_r2_xgb_group_2,
        'n_estimators': xgb_params_group_2_optimized['n_estimators'],
        'max_depth': xgb_params_group_2_optimized['max_depth'],
        'learning_rate': xgb_params_group_2_optimized['learning_rate']
    })
        # Print performance metrics for each iteration
    # 打印每次迭代的结果
    print(f"Iteration {iteration + 1} Results:")
    print("Random Forest Group 1:", rf_results_group_1[-1])
    print("Random Forest Group 2:", rf_results_group_2[-1])
    print("XGBoost Group 1:", xgb_results_group_1[-1])
    print("XGBoost Group 2:", xgb_results_group_2[-1])

# 将结果转换为DataFrame
# 将结果转化为DataFrame进行输出
xgb_results_group_1_df = pd.DataFrame(xgb_results_group_1)
xgb_results_group_2_df = pd.DataFrame(xgb_results_group_2)
rf_results_group_1_df = pd.DataFrame(rf_results_group_1)
rf_results_group_2_df = pd.DataFrame(rf_results_group_2)

# 分别保存为四个独立的Excel文件
xgb_results_group_1_df.to_excel('8.xlsx', index=False)
xgb_results_group_2_df.to_excel('88.xlsx', index=False)
rf_results_group_1_df.to_excel('888.xlsx', index=False)
rf_results_group_2_df.to_excel('8888.xlsx', index=False)
    